# Introduction

This is the first part of a python based system to analyse news headlines published around the world for signs of epidemics based on the kind of monitoring the World Health Organisation (WHO) conducts. This first phase of this program as demonstrated in this Python script does the following:

* Reads a test headlines file line by line
* Identifies relevant news headlines
* Extracts the locations (city and/or country name) from each news headline.
* Adds the data to a panda DataFrame.

This script uses 3 alternatives for extracting location names:

1. A regular exression only
2. A local instance of MySQL which has had the geonames data installed
3. The Geonames services which is accessed using a REST based API call


# Obtaining Geonames Data

The geonames dataset is used to provide a respository of location names from around the world.  You can access Geonames data using the geonamescahce library. If using anaconda the conda import for this data is not compatable with python 3 and so the following pip install needs to be used to install the library *pip install geonamescache*. Similar comments apply to the unidecode library.

Use of geonamescache may hit constraints in Jupyter Notebook. You can adjust some Jupyter Notebook parameters or consider two alternative approaches:

* use the geonames webservices REST API or 
* doanload the geonames data and instantiate a local database. This is easy to do and well documented online.

To access the online webservices API use the unencrypted URI http://api.geonames.org/search which is provided below in the *genamesapi* variable in the parameters cell. Accessing the online service is easy and efficient. There are two versions: a free version and a paid for version. The free version limits the number of requests to 1000 per hour. If you exceed this limit your requests are throttled. The paid for service does not have this constraint, but then its a paid for service!

To access the data locally I found the easiest way to do this was to add the data to a local database, in my case a MySQL database. The instructions are given in http://codigofuerte.github.io/GeoNames-MySQL-DataImport/downloading_data.html

The processes is quite straight forward:
* download the files from the above github repro above (geonames_importer.sh, admin1CodesASCII.txt, allCountries.txt etc)
* download the data using the script (geonames_importer.sh)
* startup your MySQL server (I uses the Bitnami WAMP deployment and MySQL Workbench)
* create thedatabase using the script (geonames_importer.sh)
* import the data using the script (geonames_importer.sh)

The download script will unzip the original files downloaded from geonames in a folder called "data". This folder will contain all the data needed for the "import-dump" action which populates the database.

The local data version of the geonames interface developed in this nootebook uses the local MySQL database to emulate the same kind of interface that is provided by the web service.



In [62]:
import re
import pandas as pd
import logging
import geonamescache
import unidecode
import requests
import json
from enum import Enum
import mysql.connector
from mysql.connector import Error

# Parameters

Global parameters


In [63]:
log_file = 'headlines.log'                       # Log file location and name
headlines_file = "../data/headlines.txt"         # Test headlines files location
log_level = logging.INFO                         # Log level to generate log out for
sequence_length = 5                              # Sequence of words to check as location name
geonamesapi = 'http://api.geonames.org/search'   # geonames API URI
class LocationData(Enum):                        # Options for location data: 
    WEB_SERVICES = 1                             # - Use geonames web service
    LOCAL_DB = 2                                 # - Use local db with geonames data
    REGEXP = 3                                   # - Use regexp
location_data = LocationData.LOCAL_DB        # Selected source of location data

#################################################################################
# Create a personal user name on geonames to use the web api                    #
username = 'kwatkins973'                        
#################################################################################

dbhost="localhost"                               # Host for database
dbport=3306                                      # port number
dbname="geonames"                                # database name
dbusername="root"                                # atabase user id
dbpassword="abidex123"                           # password for user


# Setup

Various setup values to allow either a local (via local database) or remote (via web service APIS calls|) geonames version of the news parser implementation.


In [64]:

###################################################################
# Terms used to identify relevant news headlines                  #
###################################################################
illnesses = ['Zika', 'Flu', 'Mad Cow', 'Meningitis', 'Cholera', 'Measles', 
             'Gonorrhea', 'Rabies', 'Malaria', 'Respiratory Syncytial', 'Bronchitis',
             'Hepatitis', 'Dengue', 'Influenza', 'Ebola', 'Tuberculosis', 'Norovirus',
             'HIV', 'Cholera', 'Pneumonia', 'Rhinovirus', 'Rotavirus', 'Mumps', 'Gonorrhea',
             'Syphilis', 'Chikungunya', 'Chickenpox', 'Chlamydia', 'Varicella', 'Varicella',
             'Herpes']
indicators = ['pandemic', 'virus', 'disease', 'eipdemic', 'vaccine']

###################################################################
# The target dataframe                                            #
###################################################################
illness_data = pd.DataFrame(columns = ['Headlines', 'Countries', 'Cities'])

###################################################################
# Logging configuration                                           #
###################################################################
logging.basicConfig(filename=log_file,
                    filemode='a',
                    level=log_level,
                    format='%(asctime)s %(levelname)s [%(filename)s:%(lineno)d] - %(message)s',
                    datefmt='%m-%d %H:%M')

###################################################################
# Geonames data to resolve location names                         #
# (cities/districts/counrties)                                    #
###################################################################
gc = geonamescache.GeonamesCache()
cities = gc.get_cities()
countries = gc.get_countries_by_names()
# print(cities) # breaks Jupyter notebook 

###################################################################
# Define punctation                                               #
# '-' , ''' explicitly removed as its in some location names      #
###################################################################
punctuation = r'!"#$%&\()*+,./:;<=>?@[\\]^_`{|}~'  
exclude = set(punctuation)


def geonames_info():
    if location_data == LocationData.WEB_SERVICES:
        return 'Getting geonames data from remote web services calls'
    elif location_data == LocationData.LOCAL_DB:
        return 'Getting geonames data from local MySQL database'
    else:
        return 'Getting geonames data using regular expression'

###################################################################
# Create  regular expressions for identfying relevant news items  #
###################################################################
def arrayToExp(values):
    """
    Convert an array of terms into a regular expression where each letter group is 
    separated with a choice operator.
    """
    exp =  ""
    exp = values[0]+exp.join(['|'+elem for elem in values[1:]])
    return exp

illness_exp = arrayToExp(illnesses)
print(illness_exp)
indicators_exp = arrayToExp(indicators)
print(indicators_exp)

illness_regexp = re.compile(illness_exp, re.IGNORECASE)
indicators_regexp = re.compile(indicators_exp, re.IGNORECASE)

# Get location data
locations_regexp = re.compile('in\s+(((st(\.)?|key|saint|north|south|east|west|'
                              'san|palm|la|new|sao|santa|los|las|rio( de)?|'
                              'santos|fort|hong|bella|oak|new)\s+)?(\w+))', re.IGNORECASE)
locations1_regexp = re.compile('in\s+((\s+[a-zA-Z\-]+)+)', re.IGNORECASE)
locations2_regexp = re.compile('in\s+((\s+[a-zA-Z\-]+)+)', re.IGNORECASE)

###################################################################
# Open and Close connections to the local MySQL database          #
###################################################################
coonnection = ''
cursor = ''
def close_db_connection():
    global connection
    global cursor
    if (connection.is_connected()):
        logging.info("Closing connection to local MySQL")
        cursor.close()
        connection.close()
        print("MySQL connection closed")

def open_db_connection():
    global connection
    global cursor
    try:
        connection = mysql.connector.connect(host     = dbhost,
                                             database = dbname,
                                             user     = dbusername,
                                             port     = dbport,
                                             password = dbpassword)
        if connection.is_connected():
            logging.info("Opening connection to local MySQL")
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
       
        return
        
    except Error as e:
        print("Error while connecting to MySQL", e)
        logging.error("Cannot connect to local MySQL: " + e)
        if (connection.is_connected()):
            close_db_connection()
            logging.error("Finally: MySQL connection closed")


Zika|Flu|Mad Cow|Meningitis|Cholera|Measles|Gonorrhea|Rabies|Malaria|Respiratory Syncytial|Bronchitis|Hepatitis|Dengue|Influenza|Ebola|Tuberculosis|Norovirus|HIV|Cholera|Pneumonia|Rhinovirus|Rotavirus|Mumps|Gonorrhea|Syphilis|Chikungunya|Chickenpox|Chlamydia|Varicella|Varicella|Herpes
pandemic|virus|disease|eipdemic|vaccine


# Parse Headlines

Loop through the headlines file and extract those headlines that look like they relate to news items about deseases.


In [65]:
def query_db(name: str, country_hint):
    global connection
    sql_select_query = "select g.name, g.latitude, g.longitude, c.name from geoname g inner join countryinfo c on c.iso_alpha2 = g.country where g.name = '" + name + "'"
    cursor = connection.cursor()
    cursor.execute(sql_select_query)
    records = cursor.fetchall()
    logging.info("Matched " + str(cursor.rowcount) + " records to : " + name)

    if len(records) > 0:
        countries = []
        for row in records:
            if row[3] not in countries:
                countries.append(row[3])
        return countries
    else:
        return ['unknown']

def get_geonames(name, country_hint):
    '''
    Returns a geo name that has been matched by the geonames web service along with the associated
    country. 
    A 'countryHint' parameter provides help in resolving ambiguities where there are more than one 
    location in the geonames database matching the parameter 'name'. If several names in geonames 
    match the name parameter then all countries containing those locations are captured. 
    If one is the same as the countryHint  then this is assumed to be the correct country otherwise 
    the first country in the list is returned and countryHint is assumed to be wrong.
    if there are no matching names then ('unknown', 'unkown') is returned for name and country.
    If something whent wrong with the request then ('Invalid', 'Invalid') is returned.

    Parameters
    ----------
    name : string
        The name or a sequence of names being matched to the geonames repositorry.
    country_hint : string
        Used as the name of the country in order to resolve ambiguities and 
        asumed to be determined from the context.

    Returns
    -------
    string
        The name of the City when a match is made or 'Unknown' in the absence 
        of a match or 'Invalid' when an error has been found.
    string
        The name of the corresponding country r 'Unknown' in the absence 
        of a match or 'Invalid' when an error has been found.
    '''
    if name in illnesses:
        return 'Unknown', 'Unknown'
    
    response = ''
    countries = []
    if location_data == LocationData.WEB_SERVICES:
        # use the geonames web service
        params = { 'q': name,
                   'maxrows': '100',
                   'username': username,
                   'type': 'json'
                  }
        resp = requests.get(geonamesapi, params=params)
        if resp.status_code != 200:
            # This means something went wrong.
            logging.error("Access error (HTTP Status code: " + 
                          resp.status_code + 
                          ") searching for: " + name)
            return 'Invalid', 'Invalid'
        
        response = json.loads(resp.text) 
        # Extract city and country
        try:    
            for names in response['geonames']:
                if (name == names['name']):
                    countries.append(names['countryName'])              
        except KeyError:
            logging.error("KeyError detected searching for: " + name)
            return 'Unknown', 'Unknown'
    else:
        # use local database
        countries = query_db(name, country_hint) 

    if len(countries) > 0:
        if country_hint in countries:
            return name, country_hint
        else:
            return name, countries[0]
    return 'Unknown', 'Unknown'


def purge_punctuation(s):
    '''
     Return the string passed in as a parameter with the punctuation removed

    Parameters
    ----------
    s : string
        The string to be remoeved.

    Returns
    -------
    string
        The input string with the punctation removed.
    '''
    return ''.join(ch for ch in s if ch not in exclude)
    

def check_using_geonames(words, country_hint, reverse = False):
    '''
    Check whether the sequence of words corresponds to a location using geonames.
    Choose the maximal sequence of words that corresepnds to a location.

    Parameters
    ----------
    words : List of strings
        The news headline as a list of strings.
    country_hint : string
        A country identifier when it can be obtained from the context to 
        resolve ambiguities.
    reverse : boolean, optional
        For processing word sequences from the end of the news headline. The default is False.

    Returns
    -------
    A place : string
        The name of a place (typically a city).
    A country : string
        The name of the country containing the city.

    '''
    place = ''
    previous_place = 'Unknown'
    previous_country = 'Unknown'
    
    found = False
    if reverse:
        res = words[::-1]
        words = res
    for i in range(len(words)):
        if reverse:
            place = words[i] + place
        else:
            place = place + words[i]
        found_place, found_country = get_geonames(place, country_hint)
        if found and (found_place == 'Unknown' or found_place == 'Invalid'):
            return previous_place, previous_country
        else:
            if found_place != 'Unknown' and found_place != 'Invalid' :
                found = True
            previous_place = found_place
            previous_country = found_country
            if reverse:
                # Work backwards when at the end of a news headline
                place = ' ' + place
            else:
                place = place + ' '
    return previous_place, previous_country


def get_location_using_geonames(text: str, country_hint):
    '''
    Add the headline to the Headlines panda if it matches location data from the geonames
    web service.
    In the case that the headline does not match the illness regular expressions it is
    retained in a log file which leaves open the possiblity of fine-tuning the 
    search criteria to catch edge cases.

    Parameters
    ----------
    text : string
        A news headline.
    country_hint : string
        A country identifier when it can be obtained from the context to 
        resolve ambiguities. The default is 'Unknown'.

    Returns
    -------
    A place : string
        The name of a place (typically a city).
    A country : string
        The name of the country containing the city.

    '''
    words = text.split() #split the sentence into individual words

    # If there is the word 'in' in the text then assume the words after refer to a location
    # take up to sequence_length of those words to match to a location in the geonames database
    if 'in' in words:
        pos = words.index('in')
        pos = pos + 1
        if len(words) > pos + sequence_length - 1:
            last_pos = pos + sequence_length
        else:
            last_pos = len(words)
        # check sequence of words from pos to max_pos against geonames
        place, country = check_using_geonames(words[pos:last_pos], country_hint)
        if place != 'Invalid' and place != 'Unknown':
            return place, country

    # Check whether the first few words correspond to a location
    pos = 0
    if len(words) > pos + sequence_length - 1:
        last_pos = pos + sequence_length
    else:
        last_pos = len(words)
    place, country = check_using_geonames(words[pos:last_pos], country_hint)
    if place != 'Invalid' and place != 'Unknown':
        return place, country
    
    # Check whether the last few names correspond to a location
    last_pos = len(words)
    if len(words) > sequence_length - 1:
        pos = last_pos - sequence_length
    else:
        pos = 0
    place, country = check_using_geonames(words[pos:last_pos], country_hint, 
                                          reverse = True)
    if place != 'Invalid' and place != 'Unknown':
        return place, country
    else:
        logging.warning("Location not found in geonames: " + text)
        return 'Unknown', 'Unknown'


def get_location(text: str, country_hint: str):
    '''
    USe a regular expression to identify candidate locations.
    
    Parameters
    ----------
    text : string
        A news headling.
    country_hint : string, optional
        A country identifier when it can be obtained from the context to 
        resolve ambiguities. The default is 'Unknown'.

    Returns
    -------
    A place : string
        The name of a place (typically a city).
    A country : string
        The name of the country containing the city.
    '''
    text = purge_punctuation(text)
    if location_data == LocationData.REGEXP:
        locations = locations_regexp.search(text)
        if locations:
            return locations.group(1), 'Unknown'
        else:
            return 'Unknown', 'Unknown'
    else:
        return get_location_using_geonames(text, country_hint)
    

def add_headline(text: str, country_hint = 'Unknown'):
    '''
    Add the headline to the Headlines panda if it matches the regular expression.
    The regualar expression is split into two kinds:
    - exact match based on an explict list of deseases and viruses
    - a separate match based on more generic indicators such as 'virus' on the assumption
      that increasing the probability of a true match will require more sophisticated
      linguistic analysis.
    In the case that the headline does not match the illness regular expressions it is
    retained in a log file which leaves open the possiblity of fine-tuning the 
    search criteria to catch edge cases.

    Parameters
    ----------
    text : str
        A news headline.
    country_hint : string, optional
        A country identifier when it can be obtained from the context to 
        resolve ambiguities. The default is 'Unknown'.

    Returns
    -------
    None.

    '''
    global illness_data
    utext = unidecode.unidecode(text)
    if illness_regexp.search(utext):
        place, country = get_location(utext, country_hint)
        illness_data = illness_data.append({"Headlines" : [text],
                                            "Countries" : [country],
                                            "Cities" : [place]}, ignore_index=True)
    elif indicators_regexp.search(text):  #May need to make the analysis more sophisticated
        place, country = get_location(utext, country_hint)
        illness_data = illness_data.append({"Headlines" : [text],
                                            "Countries" : [country],
                                            "Cities" : [place]}, ignore_index=True)
    else:
        logging.warning("No disease reference: " + text)
        

def read_hedlines_file(filename):
    """
    Read a headlines file and check whether the headline relates to illnesses/deseases of interest.
    """
    print("Reading lines from " + filename)
    file = open(filename, 'r')
    if location_data == LocationData.LOCAL_DB:
        open_db_connection()
    for line in file:
        add_headline(line.strip())
    if location_data == LocationData.LOCAL_DB:
        close_db_connection()
    file.close()

def main():
    read_hedlines_file(headlines_file)
    print(illness_data)
    
if __name__ == '__main__':
    main()


Reading lines from ../data/headlines.txt
Connected to MySQL Server version  5.7.21
MySQL connection closed
                                        Headlines   Countries        Cities
0  [ï»¿More Zika patients reported in Fort Worth]  [Zimbabwe]  [Fort Worth]
